## Import libraries

In [1]:
import pandas as pd
import lizard 


## Read data

In [2]:
data = pd.read_csv("./data/gh_apache_!_accumulo.csv")

In [3]:
data.head()


,project,url,discussion,pull_number,pull_id,filename,start_line,original_start_line,start_side,side,...,updated_at,commit_before,commit_while,commit_after,url_source_before,url_source_while,url_source_after,file_content_before,file_content_while,file_content_after
0,apache/accumulo,https://github.com/apache/accumulo/pull/1622,https://github.com/apache/accumulo/pull/1622#d...,1622,428010670,core/src/main/java/org/apache/accumulo/core/co...,NaN,NaN,NaN,RIGHT,...,2020-06-05T12:44:03Z,dbfab01a39e7bc6ac92f8641fc4b1c0f0c22039f,dbfab01a39e7bc6ac92f8641fc4b1c0f0c22039f,4d6991afc9c9589b829c2b9677a47e85586cc5aa,https://raw.githubusercontent.com/apache/accum...,https://raw.githubusercontent.com/apache/accum...,https://raw.githubusercontent.com/apache/accum...,/*\n * Licensed to the Apache Software Foundat...,/*\n * Licensed to the Apache Software Foundat...,/*\n * Licensed to the Apache Software Foundat...
1,apache/accumulo,https://github.com/apache/accumulo/pull/1622,https://github.com/apache/accumulo/pull/1622#d...,1622,428010670,core/src/main/java/org/apache/accumulo/core/co...,NaN,NaN,NaN,RIGHT,...,2020-06-05T12:44:03Z,dbfab01a39e7bc6ac92f8641fc4b1c0f0c22039f,dbfab01a39e7bc6ac92f8641fc4b1c0f0c22039f,4d6991afc9c9589b829c2b9677a47e85586cc5aa,https://raw.githubusercontent.com/apache/accum...,https://raw.githubusercontent.com/apache/accum...,https://raw.githubusercontent.com/apache/accum...,/*\n * Licensed to the Apache Software Foundat...,/*\n * Licensed to the Apache Software Foundat...,/*\n * Licensed to the Apache Software Foundat...
2,apache/accumulo,https://github.com/apache/accumulo/pull/1622,https://github.com/apache/accumulo/pull/1622#d...,1622,428010670,core/src/main/java/org/apache/accumulo/core/co...,NaN,NaN,NaN,RIGHT,...,2020-06-05T12:44:03Z,dbfab01a39e7bc6ac92f8641fc4b1c0f0c22039f,dbfab01a39e7bc6ac92f8641fc4b1c0f0c22039f,4d6991afc9c9589b829c2b9677a47e85586cc5aa,https://raw.githubusercontent.com/apache/accum...,https://raw.githubusercontent.com/apache/accum...,https://raw.githubusercontent.com/apache/accum...,/*\n * Licensed to the Apache Software Foundat...,/*\n * Licensed to the Apache Software Foundat...,/*\n * Licensed to the Apache Software Foundat...
3,apache/accumulo,https://github.com/apache/accumulo/pull/1622,https://github.com/apache/accumulo/pull/1622#d...,1622,428010670,core/src/main/java/org/apache/accumulo/core/co...,NaN,NaN,NaN,RIGHT,...,2020-06-05T12:44:03Z,dbfab01a39e7bc6ac92f8641fc4b1c0f0c22039f,dbfab01a39e7bc6ac92f8641fc4b1c0f0c22039f,4d6991afc9c9589b829c2b9677a47e85586cc5aa,https://raw.githubusercontent.com/apache/accum...,https://raw.githubusercontent.com/apache/accum...,https://raw.githubusercontent.com/apache/accum...,/*\n * Licensed to the Apache Software Foundat...,/*\n * Licensed to the Apache Software Foundat...,/*\n * Licensed to the Apache Software Foundat...
4,apache/accumulo,https://github.com/apache/accumulo/pull/1614,https://github.com/apache/accumulo/pull/1614#d...,1614,420303336,test/src/main/java/org/apache/accumulo/test/fu...,NaN,NaN,NaN,RIGHT,...,2020-06-04T19:21:57Z,13ad1b33bc3e92669db8a523e1e3a87025326d7a,ab92f60d0687aa9e3be8b0637240c088739b71fa,6319d9e408350cd15570eb9a93de613e5d4e730e,https://raw.githubusercontent.com/apache/accum...,https://raw.githubusercontent.com/apache/accum...,https://raw.githubusercontent.com/apache/accum...,/*\n * Licensed to the Apache Software Foundat...,/*\n * Licensed to the Apache Software Foundat...,/*\n * Licensed to the Apache Software Foundat...


## Data preprocessing

In [4]:
def extract_function_body(file_content, start_line, end_line):
    """
    extract the function body from the file content
    """
    lines = file_content.splitlines()
    # -1 to exclude the last line }
    function_body = lines[start_line:end_line-1]
    # concat the lines
    function_body = " ".join(function_body)
    return function_body


In [5]:
def extract_function(file_content_type):
    extracted_functions = []
    # loop through the rows. data is the dataframe that defined above
    for index, row in data.iterrows():
        file_content = row[file_content_type]
        # provide the file content to lizard to extract the functions, the function needs a filename to work so we just use the filename
        parse_result = lizard.analyze_file.analyze_source_code(row['filename'], file_content)
        functions = parse_result.function_list
        for function in functions:
            # from the file content, extract the function body using the start and end line
            function_body = extract_function_body(file_content, function.start_line, function.end_line)
            method = function.long_name + "{" + function_body + "}"
            function_data = {
                "method_name": function.long_name,
                "method_body": function_body,
                "method": method,
                "start_line": function.start_line,
                "end_line": function.end_line,
            }
            extracted_functions.append(function_data)
            
        # break to only loop the first row. TODO remove this 
        break
    return extracted_functions




In [6]:
extracted_functions_while = extract_function('file_content_while')
extracted_functions_after = extract_function('file_content_after')
print(extracted_functions_while)


[{'method_name': 'Property( String name , String defaultValue , PropertyType type , String description)', 'method_body': '    this.key = name;     this.defaultValue = defaultValue;     this.description = description;     this.type = type;', 'method': 'Property( String name , String defaultValue , PropertyType type , String description){    this.key = name;     this.defaultValue = defaultValue;     this.description = description;     this.type = type;}', 'start_line': 983, 'end_line': 988}, {'method_name': 'toString()', 'method_body': '    return this.key;', 'method': 'toString(){    return this.key;}', 'start_line': 991, 'end_line': 993}, {'method_name': 'getKey()', 'method_body': '    return this.key;', 'method': 'getKey(){    return this.key;}', 'start_line': 1000, 'end_line': 1002}, {'method_name': 'getDefaultValue()', 'method_body': '    return this.defaultValue;', 'method': 'getDefaultValue(){    return this.defaultValue;}', 'start_line': 1010, 'end_line': 1012}, {'method_name': '